# 提出问题

这里我们主要对以下几个业务指标进行分析：

① 月均消费次数

② 月均消费金额

③ 客单价

④ 消费趋势

---

# 理解数据

朝阳医院的药品数据来自于社群，具体采集数据的方式有很多种，比如：网络爬虫、数据库提取数据、埋点采集数据等等，这里就不细说了。

In [1]:
# 导入数据分析包
import pandas as pd

In [2]:
# 读取Excel数据，此处统一先按照对象类型读入，之后再转换
filenameStr = r'C:\Users\Richard\Desktop\跟猴子学习人工智能核心技术\数据分析中级（Python）\第4关 数据分析的基本过程\朝阳医院2018年销售数据.xlsx'
xlsx = pd.ExcelFile(filenameStr, dtype = 'object')
salesDf = xlsx.parse('Sheet1', dtype = 'object')

In [3]:
# 打印出前5行，以确保数据正常运行
salesDf.head()

,购药时间,社保卡号,商品编码,商品名称,销售数量,应收金额,实收金额
0,2018-01-01 星期五,001616528,236701,强力VC银翘片,6,82.8,69
1,2018-01-02 星期六,001616528,236701,清热解毒口服液,1,28,24.64
2,2018-01-06 星期三,0012602828,236701,感康,2,16.8,15
3,2018-01-11 星期一,0010070343428,236701,三九感冒灵,1,28,28
4,2018-01-15 星期五,00101554328,236701,三九感冒灵,8,224,208


In [4]:
# 查看有多少行，多少列
salesDf.shape

(6578, 7)

In [5]:
print('有{0}行，{1}列'.format(salesDf.shape[0], salesDf.shape[1]))

有6578行，7列


In [6]:
# 查看每一列的数据类型
salesDf.dtypes

购药时间    object
社保卡号    object
商品编码    object
商品名称    object
销售数量    object
应收金额    object
实收金额    object
dtype: object

---

# 数据清洗

## 选择子集

数据集中有很多字段的数据，有时候我们只需要选择自己需要的数据进行分析就可以了。

本案例不需要选择子集，所以下面的选择子集代码被注释掉了。

In [ ]:
# 选择子集
# subsalesDf = salesDf.loc[:,'购药时间':'销售数量']

## 列名重命名

因为我们是以医院的视角去分析数据，所以这里我们需要将购药时间改为销售时间。

In [7]:
# 定义字典：旧列名和新列名对应关系
colnameStr = {'购药时间':'销售时间'}

'''
inplace=False，数据框本身不会变，而会创建一个改动后的新的数据框，
默认的inplace是False；
inplace=True，数据框本身会改动。
'''
salesDf.rename(columns=colnameStr, inplace=True)

In [8]:
# 打印出前5行，发现购药时间已经改为销售时间
salesDf.head()

,销售时间,社保卡号,商品编码,商品名称,销售数量,应收金额,实收金额
0,2018-01-01 星期五,001616528,236701,强力VC银翘片,6,82.8,69
1,2018-01-02 星期六,001616528,236701,清热解毒口服液,1,28,24.64
2,2018-01-06 星期三,0012602828,236701,感康,2,16.8,15
3,2018-01-11 星期一,0010070343428,236701,三九感冒灵,1,28,28
4,2018-01-15 星期五,00101554328,236701,三九感冒灵,8,224,208


## 缺失数据处理

原始数据集经常会由于记录错误导致缺失某些数据，我们可以采用下面两种方法来处理：

① 直接删除缺失数据；

② 如果缺失数据太多，我们可以通过建立模型进行插值的办法来补充这些数据。

这里我们采用第1种方法

In [9]:
print('删除缺失值前大小：', salesDf.shape)

删除缺失值前大小： (6578, 7)


我们需要删除销售时间或者社保卡号列中为空的行：

In [10]:
# how='any'表示在给定的任何一列中有缺失值就删除该行
salesDf = salesDf.dropna(subset=['销售时间','社保卡号'], how='any')

print('删除缺失值后大小：',salesDf.shape)

删除缺失值后大小： (6575, 7)


## 数据类型转换

因为我们在导入Excel数据的时候统一按照字符串格式读取了数据，现在就需要将某些列的字符串数据类型转换为需要的数据类型。

显然，我们需要将销售数量、应收金额、实收金额列的数据类型转换为数值类型（浮点型）：

In [11]:
print('转换前的数据类型：\n', salesDf.dtypes)

转换前的数据类型：
 销售时间    object
社保卡号    object
商品编码    object
商品名称    object
销售数量    object
应收金额    object
实收金额    object
dtype: object


In [12]:
# 字符串转换为数值类型（浮点型）
salesDf['销售数量'] = salesDf['销售数量'].astype('float')
salesDf['应收金额'] = salesDf['应收金额'].astype('float')
salesDf['实收金额'] = salesDf['实收金额'].astype('float')

print('转换后的数据类型：\n', salesDf.dtypes)

转换后的数据类型：
 销售时间     object
社保卡号     object
商品编码     object
商品名称     object
销售数量    float64
应收金额    float64
实收金额    float64
dtype: object


除此之外，我们还需要将销售时间这一列的字符串转换为日期时间数据类型。

这里的销售时间为“日期+星期”的格式，我们只需要日期就可以了，这里就要用到字符串的分割。

因为我们要处理的销售时间这一列数据量很大，可以定义一个函数进行批量分割：

In [13]:
'''
定义函数：分割销售时间，获取销售日期
输入：timecolSer 销售时间这一列，是个Series数据类型
输出：分割后的时间，返回也是一个Series数据类型
'''
def splitSaledate(timecolSer):
    dateList = []
    
    for value in timecolSer:
        dateStr = value.split(' ')[0]
        dateList.append(dateStr)
    
    # 将列表转化为一维数据Series类型
    dateSer = pd.Series(dateList)
    
    return dateSer

In [14]:
# 获取“销售时间”这一列
timecolSer = salesDf.loc[:,'销售时间']
# 对字符串进行分割，获取销售时间
dateSer = splitSaledate(timecolSer)

In [15]:
dateSer[0:3]

0    2018-01-01
1    2018-01-02
2    2018-01-06
dtype: object

In [16]:
# 修改销售时间这一列的值
salesDf.loc[:,'销售时间'] = dateSer

In [17]:
salesDf.head()

,销售时间,社保卡号,商品编码,商品名称,销售数量,应收金额,实收金额
0,2018-01-01,001616528,236701,强力VC银翘片,6.0,82.8,69.00
1,2018-01-02,001616528,236701,清热解毒口服液,1.0,28.0,24.64
2,2018-01-06,0012602828,236701,感康,2.0,16.8,15.00
3,2018-01-11,0010070343428,236701,三九感冒灵,1.0,28.0,28.00
4,2018-01-15,00101554328,236701,三九感冒灵,8.0,224.0,208.00


下面我们需要把销售时间字符串类型转换为日期时间类型：


In [18]:
# errors='coerce'：如果原始数据不符合日期的格式，转换后的值为空值NaT
# format：是原始数据中的日期格式
salesDf.loc[:,'销售时间'] = pd.to_datetime(salesDf.loc[:,'销售时间'],
                                           format='%Y-%m-%d',
                                           errors='coerce')

In [19]:
salesDf.dtypes

销售时间    datetime64[ns]
社保卡号            object
商品编码            object
商品名称            object
销售数量           float64
应收金额           float64
实收金额           float64
dtype: object

转换日期过程中不符合日期格式的数值会被转换为空值，这里需要继续删除列（销售时间，社保卡号）中为空的行。

In [21]:
print('删除缺失值前大小：',salesDf.shape)

salesDf = salesDf.dropna(subset=['销售时间','社保卡号'], how='any')

print('删除缺失值后大小：',salesDf.shape)

删除缺失值前大小： (6575, 7)
删除缺失值后大小： (6549, 7)


## 数据排序

In [22]:
print('排序前的数据集')
salesDf

排序前的数据集


,销售时间,社保卡号,商品编码,商品名称,销售数量,应收金额,实收金额
0,2018-01-01,001616528,236701,强力VC银翘片,6.0,82.8,69.00
1,2018-01-02,001616528,236701,清热解毒口服液,1.0,28.0,24.64
2,2018-01-06,0012602828,236701,感康,2.0,16.8,15.00
3,2018-01-11,0010070343428,236701,三九感冒灵,1.0,28.0,28.00
4,2018-01-15,00101554328,236701,三九感冒灵,8.0,224.0,208.00
5,2018-01-20,0013389528,236701,三九感冒灵,1.0,28.0,28.00
6,2018-01-31,00101464928,236701,三九感冒灵,2.0,56.0,56.00
7,2018-02-17,0011177328,236701,三九感冒灵,5.0,149.0,131.12
8,2018-02-22,0010065687828,236701,三九感冒灵,1.0,29.8,26.22
9,2018-02-24,0013389528,236701,三九感冒灵,4.0,119.2,104.89


In [23]:
# 按销售日期进行升序排序
salesDf = salesDf.sort_values(by='销售时间',
                             ascending=True)

In [24]:
print('排序后的数据集')
salesDf

排序后的数据集


,销售时间,社保卡号,商品编码,商品名称,销售数量,应收金额,实收金额
0,2018-01-01,001616528,236701,强力VC银翘片,6.0,82.8,69.0
3436,2018-01-01,0010616728,865099,硝苯地平片(心痛定),2.0,3.4,3.0
1190,2018-01-01,0010073966328,861409,非洛地平缓释片(波依定),5.0,162.5,145.0
3859,2018-01-01,0010073966328,866634,硝苯地平控释片(欣然),6.0,111.0,92.5
3888,2018-01-01,0010014289328,866851,缬沙坦分散片(易达乐),1.0,26.0,23.0
894,2018-01-01,0013331728,861405,苯磺酸氨氯地平片(络活喜),2.0,69.0,62.0
893,2018-01-01,0011743428,861405,苯磺酸氨氯地平片(络活喜),1.0,34.5,31.0
4368,2018-01-01,00103283128,870921,卡托普利片,1.0,2.4,2.2
4562,2018-01-01,0010074599128,874684,厄贝沙坦氢氯噻嗪片(依伦平),5.0,118.0,118.0
5039,2018-01-01,0010017493928,868042,马来酸左旋氨氯地平片(玄宁),1.0,46.0,46.0


In [25]:
# 重命名行索引（index）：排序后的行索引是之前的行号，需要修改为从0到N按顺序的行索引
salesDf = salesDf.reset_index(drop=True)

In [26]:
salesDf

,销售时间,社保卡号,商品编码,商品名称,销售数量,应收金额,实收金额
0,2018-01-01,001616528,236701,强力VC银翘片,6.0,82.8,69.0
1,2018-01-01,0010616728,865099,硝苯地平片(心痛定),2.0,3.4,3.0
2,2018-01-01,0010073966328,861409,非洛地平缓释片(波依定),5.0,162.5,145.0
3,2018-01-01,0010073966328,866634,硝苯地平控释片(欣然),6.0,111.0,92.5
4,2018-01-01,0010014289328,866851,缬沙坦分散片(易达乐),1.0,26.0,23.0
5,2018-01-01,0013331728,861405,苯磺酸氨氯地平片(络活喜),2.0,69.0,62.0
6,2018-01-01,0011743428,861405,苯磺酸氨氯地平片(络活喜),1.0,34.5,31.0
7,2018-01-01,00103283128,870921,卡托普利片,1.0,2.4,2.2
8,2018-01-01,0010074599128,874684,厄贝沙坦氢氯噻嗪片(依伦平),5.0,118.0,118.0
9,2018-01-01,0010017493928,868042,马来酸左旋氨氯地平片(玄宁),1.0,46.0,46.0


## 异常值处理

In [27]:
# 查看描述统计信息
salesDf.describe()

,销售数量,应收金额,实收金额
count,6549.000000,6549.000000,6549.000000
mean,2.384486,50.449076,46.284370
std,2.375227,87.696401,81.058426
min,-10.000000,-374.000000,-374.000000
25%,1.000000,14.000000,12.320000
50%,2.000000,28.000000,26.500000
75%,2.000000,59.600000,53.000000
max,50.000000,2950.000000,2650.000000


从上面的描述统计信息中，我们发现销售数量存在负数，这明显不符合逻辑，需要进行异常值的处理。


In [28]:
# 删除异常值：通过条件判断筛选出数据

#1）定义查询条件
querySer = salesDf.loc[:,'销售数量'] > 0

In [29]:
querySer

0       True
1       True
2       True
3       True
4       True
5       True
6       True
7       True
8       True
9       True
10      True
11      True
12      True
13      True
14      True
15      True
16      True
17      True
18      True
19      True
20      True
21      True
22      True
23      True
24      True
25      True
26      True
27      True
28      True
29      True
        ... 
6519    True
6520    True
6521    True
6522    True
6523    True
6524    True
6525    True
6526    True
6527    True
6528    True
6529    True
6530    True
6531    True
6532    True
6533    True
6534    True
6535    True
6536    True
6537    True
6538    True
6539    True
6540    True
6541    True
6542    True
6543    True
6544    True
6545    True
6546    True
6547    True
6548    True
Name: 销售数量, Length: 6549, dtype: bool

In [31]:
type(querySer)

pandas.core.series.Series

In [32]:
# 2）应用查询条件
print('删除异常值前：', salesDf.shape)

salesDf = salesDf.loc[querySer,:]

print('删除异常值后：', salesDf.shape)

删除异常值前： (6549, 7)
删除异常值后： (6506, 7)


---

# 构建模型

## KPI 1：月均消费次数

> **月均消费次数 = 总消费次数 / 月份数**

① 总消费次数

总消费次数：同一天内，同一个人发生的所有消费算作一次消费。

我们需要根据列名（销售时间，社保卡号）判断，如果这两个列值同时相同，只保留1条即可，将重复的数据删除。

step 1：删除重复数据

In [ ]:
kpi1_Df = salesDf.drop_duplicates(subset=['销售时间','社保卡号'])

step 2：查看行数（总消费次数）


In [ ]:
totalⅠ = kpi1_Df.shape[0]
print('总消费次数 =', totalⅠ)

② 月份数

step 1：排序

In [ ]:
# 按销售时间升序排序
kpi1_Df = kpi1_Df.sort_values(by='销售时间',
                             ascending=True)
# 重命名行名（index）
kpi1_Df = kpi1_Df.reset_index(drop=True)

In [ ]:
kpi1_Df.head()

step 2：获取时间范围

In [ ]:
# 最小时间值
startTime = kpi1_Df.loc[0, '销售时间']
# 最大时间值
endTime = kpi1_Df.loc[totalI - 1, '销售时间']

step 3：计算月份数

In [ ]:
# 天数
daysⅠ = (endTime - startTime).days
# 月份数
monthsⅠ = daysⅠ / 30

print('月份数为：',monthsⅠ)

③ 业务指标1：月均消费次数 = 总消费次数 / 月份数

In [ ]:
kpi1_Ⅰ = totalI / monthsⅠ

print('业务指标1：月均消费次数 =', kpi1_Ⅰ)

## KPI 2：月均消费金额

> **月均消费金额 = 总消费金额 / 月份数**

In [ ]:
# 总消费金额
totalMoneyF = salesDf.loc[:,'实收金额'].sum()
# 月均消费金额
monthMoneyF = totalMoneyF / monthsⅠ

print('业务指标2：月均消费金额 =', monthMoneyF)

## KPI 3：客单价

> **客单价 = 总消费金额 / 总消费次数**

In [ ]:
pct = totalMoneyF / totalⅠ

print('业务指标3：客单价 =', pct)

门店的销售金额是由客单价和顾客数所决定的。因此要提升门店的销售额，除了尽可能多的吸引顾客，增加顾客交易的次数以外，提高客单价也是非常重要的途径。